# Run notebooks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob

import papermill as pm

import util

/glade/work/mclong/miniconda3/envs/cesm-exp/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


## Get control structure

In [3]:
control = util.get_control_dict()
control

{'output_dir': './_computed-notebooks',
 'project': 'NCGD0011',
 'data_sources': {'casename': 'smyle-fosi.regional.lat-range-basin.000'},
 'book_toc': {'format': 'jb-book',
  'root': 'description',
  'parts': [{'caption': 'Ocean Carbon Cycle',
    'chapters': [{'file': 'ocn-fgco2'}]},
   {'caption': 'Ocean Tracers',
    'chapters': [{'file': 'intro-ocn-tracer-views',
      'sections': [{'glob': 'ocn-tracer-views*'}]}]}]},
 'book_config_keys': {'title': 'My CESM Experiment',
  'repository': {'url': 'https://github.com/matt-long/my-cesm-experiment',
   'path_to_book': 'notebooks',
   'branch': 'main'}},
 'compute_notebooks': {'description': {'use_cluster': False,
   'parameter_groups': {'none': {'casename': 'smyle-fosi.regional.lat-range-basin.000'}},
   'kernel_name': 'cesm-exp'},
  'ocn-fgco2': {'use_cluster': True,
   'parameter_groups': {'none': {'casename': 'smyle-fosi.regional.lat-range-basin.000',
     'component': 'pop',
     'stream': 'h'}},
   'kernel_name': 'cesm-exp'},
  'ocn

In [4]:
util.setup_book()

## Spin up a cluster

The notebooks are configured to connect to this cluster.

In [5]:
cluster = util.get_Cluster(project=control['project'])
cluster.scale(32)
cluster

## Run all the notebooks

In [6]:
output_dir = control['output_dir']

for nb, info in control['compute_notebooks'].items():
    
    parameter_groups = info['parameter_groups']
    use_cluster = info['use_cluster']
    
    default_params = {}
    if 'default_params' in info:
        default_params = info['default_params']

    for key, parms in parameter_groups.items():

        input_path = f'{nb}.ipynb'
        output_path = (
            f'{output_dir}/{nb}-{key}.ipynb'
            if key != 'none' else f'{output_dir}/{nb}.ipynb'
        )

        # check notebook expectations
        nb_api = pm.inspect_notebook(input_path)
        
        # TODO: validate paramter and API

        if nb_api:
            parms_in = dict(**default_params)
            parms_in.update(dict(**parms))
            parms_in['cluster_scheduler_address'] = cluster.scheduler_address
        else:
            parms_in = {}

        print(f'executing {input_path}')
        o = pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            kernel_name=info['kernel_name'],
            parameters=parms_in,
            engine_name='md_jinja',
            jinja_data=parms,
        )

executing description.ipynb


Executing:   0%|          | 0/2 [00:00<?, ?cell/s]

executing ocn-fgco2.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

executing ocn-tracer-views.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

executing ocn-tracer-views.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

executing ocn-tracer-views.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

In [8]:
cluster.close()